In [290]:
import time      # for testing use only
import os         # for testing use only

import bs4
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np

In [291]:
homepage = 'https://www.homeless.co.il'
page1= requests.get(homepage + '/sale/')
soup = BeautifulSoup(page1.text, 'html.parser')
#soup.prettify()



In [292]:
def check_if_exist(detail):
    if (detail == "/Images/uncheked.png"):
         exist = 0 #not exist
    else:
        exist = 1 #exist
    return exist

In [293]:
def chek_num_of_floor(floor):
    if (len(floor) > 0):
        if (floor[0] >= '0' and floor[0] <= '9'):
            return floor[:2]
        elif (floor[0] == 'מ'):
            return '-1'
        elif (floor[0] == 'ק'):
            return '0'
        else:
            return None
    else:
        return None

In [294]:
def chek_url(url):
    This_Page = requests.get(url)
    this_soup = BeautifulSoup(This_Page.text, 'html.parser')
    if (this_soup.find('h1').get_text() == "מה עושים בהומלס?"):
        return 1
    else:
        return 0

In [297]:
type_house=[] #סוג הנכס
city=[] # עיר
street=[] #רחוב
price=[] #מחיר
num_of_rooms=[] #מספר חדרים
floor=[] #קומה
elevator=[] #מעלית
furniture=[] #רהיטים
bars=[] #סורגים
terrace=[] #מרפסת
parking=[] #יש חניה או אין
size_per_mm =[] #גודל הדירה במ"מ
n_pages =266
i=0
bar=0
main_data =soup.find_all('tr',attrs={'type':'ad'})
j=0
count_pages = 0
count = 1
for page in range(n_pages):
    count_pages = count_pages +1
    for lst in main_data:
        count = count+1
        all_apartment_pages= lst.find('a', attrs={'target':'_blank'} )
        html_doc = homepage + all_apartment_pages.get("href")
        if (chek_url(html_doc) == 1):
            i = i +1
            continue
        apartment_page = requests.get(html_doc)
        details_soup = BeautifulSoup(apartment_page.text, "html.parser")
        details=details_soup.find_all('div',style=lambda value: value and 'float:right' in value and ('width:130px;' in value or 
                                                                                             'width:150px;' in value or
                                                                                             'width:180px;' in value))

        string_details = [de.find('img') for de in details]
        bars.append(check_if_exist(string_details[0].get("src"))) #bars - 1 / 0
        furniture.append(check_if_exist(string_details[1].get("src"))) #furniture - 1 / 0
        elevator.append(check_if_exist(string_details[2].get("src"))) #elevator - 1 / 0
        terrace.append(check_if_exist(string_details[3].get("src"))) #terrace - 1 / 0
        parking.append(check_if_exist(string_details[4].get("src"))) #parking - 1 / 0
        all_td= lst.find_all('td')
        string_td = [tag.get_text() for tag in all_td]
        type_house.append(string_td[2]) #typehouse
        price.append(string_td[8]) #price
        if(i<25):
            size_per_mm.append(details[6].find('span').get_text())
            city.append(string_td[3]) #city
            street.append(string_td[5]) #street
            num_of_rooms.append(string_td[6]) #ROOMS
            floor.append(string_td[7]) #floor
        else:
            if(i>=25 and i<45):
                size_per_mm.append(details[7].find('span').get_text())
                city.append(string_td[4]) #city
                street.append(string_td[5]) #street
                num_of_rooms.append(string_td[7]) #ROOMS
                floor.append(chek_num_of_floor(details[6].find('span').get_text())) #floor
        i=i+1
    end_block = soup.find('div', attrs={'class':'paging'})
    next_p = end_block.find('a', attrs={'class':'pagingtext'} )
    html_dock_new = homepage + next_p.get("href")
    n_page= requests.get(html_dock_new)
    soup = BeautifulSoup(n_page.text, "html.parser")
    main_data = soup.find_all('tr',attrs={'type':'ad'})
    i=0
    print(count_pages)
df=pd.DataFrame({'Type house': type_house ,  'City': city, 'Street':street, 'Num of rooms': num_of_rooms ,  'Floor': floor,
                 'Price':price,  'Bars': bars,  'Furniture': furniture,  'Elevator': elevator,  'Terrace': terrace
                ,  'Parking': parking,  'Size of house per mm': size_per_mm})
df



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266


,Type house,City,Street,Num of rooms,Floor,Price,Bars,Furniture,Elevator,Terrace,Parking,Size of house per mm
0,דירה,רמת גן,משה שרת,5,1,"3,120,000 ₪",1,0,1,0,1,114
1,דו משפחתי,רמת גן,יד שלום,4,1,"4,480,000 ₪",0,0,0,1,1,140
2,דו משפחתי,רמת ישי,פיקוס 5,6,קרקע,"2,250,000 ₪",1,0,0,1,1,180
3,דירה,רמת גן,שלם,3,1,"2,150,000 ₪",0,0,0,0,0,80
4,דירה,תל אביב יפו,הפלמ''ח,3,2,"2,500,000 ₪",1,1,0,1,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...
11962,פנטהאוז,רמת גן מגדל סיטי טאוור,פארק צמרת מגדל סיטי טאוור,3,30,"4,900,000 ₪",0,1,1,1,1,125
11963,בית פרטי,פתח תקווה,כפר גנים ב,6,0,"3,800,000 ₪",1,1,0,1,1,182
11964,וילה,פתח תקווה,בת גנים,9,0,"7,800,000 ₪",1,1,1,1,1,550
11965,פנטהאוז,פתח תקווה,אם המושבות החדשה,5,19,"3,330,000 ₪",0,1,1,1,1,160


In [298]:
df

,Type house,City,Street,Num of rooms,Floor,Price,Bars,Furniture,Elevator,Terrace,Parking,Size of house per mm
0,דירה,רמת גן,משה שרת,5,1,"3,120,000 ₪",1,0,1,0,1,114
1,דו משפחתי,רמת גן,יד שלום,4,1,"4,480,000 ₪",0,0,0,1,1,140
2,דו משפחתי,רמת ישי,פיקוס 5,6,קרקע,"2,250,000 ₪",1,0,0,1,1,180
3,דירה,רמת גן,שלם,3,1,"2,150,000 ₪",0,0,0,0,0,80
4,דירה,תל אביב יפו,הפלמ''ח,3,2,"2,500,000 ₪",1,1,0,1,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...
11962,פנטהאוז,רמת גן מגדל סיטי טאוור,פארק צמרת מגדל סיטי טאוור,3,30,"4,900,000 ₪",0,1,1,1,1,125
11963,בית פרטי,פתח תקווה,כפר גנים ב,6,0,"3,800,000 ₪",1,1,0,1,1,182
11964,וילה,פתח תקווה,בת גנים,9,0,"7,800,000 ₪",1,1,1,1,1,550
11965,פנטהאוז,פתח תקווה,אם המושבות החדשה,5,19,"3,330,000 ₪",0,1,1,1,1,160


In [299]:
df.to_csv("apartment data homeless.csv" ,encoding = 'utf-8-sig')